In [2]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), '../../../../'))
sys.path.append(work_path)
from helper.regrassion import *
from helper.plot import *
from helper.util import *
from helper.analysis import *

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV

In [3]:
origin = my_read_excel('C:/Users/Jihwan/Desktop/01Class/E.추론통계/E.InferentialStatistics/수업자료/fish.xlsx')
my_pretty_table(origin.head())

데이터프레임 크기: 행 수: 56, 열 수: 4

데이터프레임 상위 5개 행
+----+--------+--------+--------+--------+
|    |   길이 |   높이 |   두께 |   무게 |
|----+--------+--------+--------+--------|
|  0 |    8.4 |   2.11 |   1.41 |    5.9 |
|  1 |   13.7 |   3.53 |      2 |     32 |
|  2 |     15 |   3.82 |   2.43 |     40 |
|  3 |   16.2 |   4.59 |   2.63 |   51.5 |
|  4 |   17.4 |   4.59 |   2.94 |     70 |
+----+--------+--------+--------+--------+

데이터프레임 하위 5개 행
+----+--------+--------+--------+--------+
|    |   길이 |   높이 |   두께 |   무게 |
|----+--------+--------+--------+--------|
| 51 |     42 |   12.8 |   6.87 |   1100 |
| 52 |     43 |  11.93 |   7.28 |   1000 |
| 53 |     43 |  12.51 |   7.42 |   1100 |
| 54 |   43.5 |   12.6 |   8.14 |   1000 |
| 55 |     44 |  12.49 |    7.6 |   1000 |
+----+--------+--------+--------+--------+

기술통계
+------+---------+---------+---------+-------+--------+-------+--------+-------+-------+
|      |   count |    mean |     std |   min |    25% |   50% |    75% |   max |   nan |

## #02. 데이터 전처리

### [1] 데이터 스케일링

In [4]:
df = my_standard_scaler(origin, yname = '무게')
my_pretty_table(df.head())

+----+----------+----------+----------+--------+
|    |     길이 |     높이 |     두께 |   무게 |
|----+----------+----------+----------+--------|
|  0 | -2.18023 | -2.01651 | -1.89617 |    5.9 |
|  1 | -1.58743 |  -1.5187 | -1.56077 |     32 |
|  2 | -1.44203 | -1.41704 | -1.31633 |     40 |
|  3 | -1.30782 |  -1.1471 | -1.20263 |   51.5 |
|  4 |  -1.1736 |  -1.1471 | -1.02641 |     70 |
+----+----------+----------+----------+--------+


### [2] 독립, 종속 변수 분리

In [5]:
yname = '무게'
x = df.drop(yname, axis=1)
y =  df[yname]
x.shape, y.shape

((56, 3), (56,))

## #03. 하이퍼파라미터 튜닝을 통한 최적 모델 찾기

선형회귀(LinearRegression), 로지스틱회귀(LogisticRegression) 등의 모델을 제외한 다른 학습 모델들은 클래스 생성자에 전달하는 파라미터가 다양하다.

이 파라미터의 최적 조합을 찾는 과정

만약 선형회귀 모델에 `params1`, `params2`, `params3`라는 파라미터가 필요하다고 가정할 경우

```python
model = LinearRegression(n_job=-1, param1=???, params2=???, params3=???)
```

각 파라미터에 설정할 수 있는 값들을 리스트를 원소로 갖는 딕셔너리 형태로 준비하면 GridSearchCV 클래스가 파라미터를 조합할 수 있는 모든 경우의 수를 수행하고 가장 좋은 성능을 보이는 파라미터 조합을 찾아낸다.

```python
params = {
    "params1": [1, 2, 3],
    "params2": [0.1, 0.01, 0.001],
    "params3": [True, False]
}
```

여기서는 실제 LinearRegression 클래스가 `n_jobs`외의 의미있는 파라미터를 받지 않으므로 빈 딕셔너리를 준비한다.

### [1] 하이퍼 파라미터 튜닝기 생성

In [6]:
model = LinearRegression(n_jobs=-1)

params = {}

fold = 10

grid_cv = GridSearchCV(model, param_grid=params, cv=fold)

grid_cv.fit(x,y)

grid_cv.best_params_

{}

In [7]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00161562]),
 'std_fit_time': array([0.00484686]),
 'mean_score_time': array([0.00013878]),
 'std_score_time': array([0.00041635]),
 'params': [{}],
 'split0_test_score': array([-58.01302285]),
 'split1_test_score': array([-2.31476956]),
 'split2_test_score': array([-45.20722663]),
 'split3_test_score': array([-4.47723355]),
 'split4_test_score': array([-5.3595436]),
 'split5_test_score': array([-17.55200254]),
 'split6_test_score': array([0.62549205]),
 'split7_test_score': array([0.53019652]),
 'split8_test_score': array([-0.05032677]),
 'split9_test_score': array([-17.58452852]),
 'mean_test_score': array([-14.94029654]),
 'std_test_score': array([19.61930517]),
 'rank_test_score': array([1])}

### [3] 학습 성능이 높은 순으로 파라미터 조합을 확인

> 여기서는 출력되는 결과 없음

In [8]:
result_df = DataFrame(grid_cv.cv_results_['params'])
result_df
result_df['mean_test_score'] = grid_cv.cv_results_['mean_test_score']
my_pretty_table(result_df.sort_values(by='mean_test_score', ascending=False))

+----+-------------------+
|    |   mean_test_score |
|----+-------------------|
|  0 |          -14.9403 |
+----+-------------------+


In [9]:
grid_cv.best_estimator_

LinearRegression(n_jobs=-1)